# Matrikel-Nr: 2235021

# **Painting Classification**

Classify the painter with labeld images of paintings. Implementation with convolutional neural networks & transfer learning.

## **Imports**

In [16]:
import os
from PIL import Image
import numpy as np
import re
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from torchvision import models
from sklearn.preprocessing import LabelEncoder

## **Data Preprocessing**

In [10]:
img_size = (224, 224)

transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ToTensor(),
])

def load_data(data_dir):
    images = []
    labels = []

    for filename in os.listdir(data_dir):
        artist_name = re.sub(r'\d+', '', filename.strip('.jpg'))

        img_path = os.path.join(data_dir, filename)
        img = Image.open(img_path).convert('RGB')
        img = transform(img)
        images.append(img)
        labels.append(artist_name)

    images = torch.stack(images)
    return images, np.array(labels)

In [19]:
data_directory = '../data/artists/artists/resized'

images, labels = load_data(data_directory)
label_encoder = LabelEncoder()
numeric_labels = label_encoder.fit_transform(labels)

train_images, test_images, train_labels, test_labels = train_test_split(images, numeric_labels, test_size=0.2, random_state=42)
train_dataset = TensorDataset(train_images, torch.tensor(train_labels))  
test_dataset = TensorDataset(test_images, torch.tensor(test_labels)) 

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

## **CNN Model**

### Create Model

In [20]:
class ResNet(nn.Module):
    def __init__(self, num_classes):
        super(ResNet, self).__init__()
        resnet = models.resnet18(pretrained=True)  # Laden des vortrainierten ResNet-Modells
        self.features = nn.Sequential(*list(resnet.children())[:-1])  # Alle Schichten außer der letzten FC-Schicht übernehmen
        self.fc = nn.Linear(resnet.fc.in_features, num_classes)  # Neue FC-Schicht für die Klassifikation

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Hyperparameter
num_classes = len(np.unique(labels)) 
learning_rate = 0.001
num_epochs = 10

# Modellinitialisierung
model = ResNet(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

c:\Users\UserNA6153\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\UserNA6153\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Train Model

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {running_loss/len(train_loader):.4f}")

Epoch 1/10 - Loss: 2.4532
Epoch 2/10 - Loss: 1.7456
Epoch 3/10 - Loss: 1.3662
Epoch 4/10 - Loss: 1.0725


KeyboardInterrupt: 

### Evaluation des trainierten Modells

In [22]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Accuracy on test set: {accuracy:.2%}")

Accuracy on test set: 45.37%
